# Zcash Miner Statistics

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
import holoviews as hv

output_notebook()
hv.extension('bokeh')

In [ ]:
KNOWN_MINERS = {
    't1KHa9CJeCy3b9rUX2BhqkFJXSxSSrhM7LJ': 'CoinMine.pl',
    't1KstPVzcNEK4ZeauQ6cogoqxQBMDSiRnGr': 'CoinMine.pl',
    't1MGLc3pb6j6hGXe8YBZaoZBEShJysaWk3b': 'Bitclub', # From Zchain
    't1PbYh7TRXigqcdXYC1YHJS3KSBR9Yeso9W': 'Slush Pool',
    't1Py7vDFNUiAodSvcxM6hU9dwb5MTPD88sh': 'uCrypto',
    't1RwbKka1CnktvAJ1cSqdn7c6PXWG4tZqgd': 'Flypool',
    't1SaATQbzURpG1qU3vz9Wfn3pwXoTqFtTq2': 'Suprnova',
    't1SmBjm4WnDrrjPyUFSuJQTeEs1TGeWyHJx': 'Waterhole',
    't1TpMVK8PUS5xqgb9dYrmmt9D3j2b9j58Zm': 'MinerGate',
    't1Tqy2u2qgTdcVf7TRxC2KGrxwcuzfdgnSf': 'Coinotron', # From Zchain
    't1VLW9RbDW7ZJ6e56q5epSzy6tz2LZ7Rt7Q': 'MinerGate',
    't1VpYecBW4UudbGcy4ufh61eWxQCoFaUrPs': 'Flypool',
    't1WrgmW1uYpxsr2Pr4W8DnDV8ppfaKjNZaH': 'DwarfPool',
    't1XepX38RxS3o5hLioLbaNb6Fa2Y2Be55xw': 'Flypool',
    't1Xk6GeseeV8FSDpgr359yL2LmaRtUdWgaq': 'Coinotron', # From Zchain
    't1Z4fP81GQcwfpTYuRafeM1ACfyZUoNRKaB': 'Coinotron',
    't1ZJQNuop1oytQ7ow4Kq8o9if3astavba5W': 'Flypool',
    't1ZW8mX1UR2YQXafSxriuK2Bz1wxqzskGQq': 'Nicehash', # From Zchain
    't1ZYZS6ynUDbvht7vH3dMiM3rsAJ1p6EGWC': 'Antpool',
    't1Zeb8vY9XJxhoLV6Y8DcQDZ1Gvv1oALmfN': 'Zpool',
    't1aZvxRLCGVeMPFXvqfnBgHVEbi4c6g8MVa': 'F2Pool',
    't1ajyFP7GnauoDFaM8MqJx9ouQjKS3tbA7g': '2Miners.com', # SOLO pool
    't1atQdfjuctcaH9pThx9uobe4RAEfNWrZrr': 'Coinfoundry',
    't1bCmsYsBx4tU8ND8LjWECkyhxY17Huinw1': 'MiningPoolHub',
    't1cXEed6nrizBXsxYtHd7ahPrjbP3RXofmX': 'Zmine',
    't1dzbQ2bnAGwBVHZykfKYTQYyZY1AjrS1Mg': '2Miners.com', # PPLNS pool
    't1emzuNbemjqnEhEue74NL3BxsR4cA1ajfP': 'Nanopool', # In their default claymore config
    't1godRk4oKt689om4Q3CbmA9wsuhm1rUvLR': 'ViaBTC',
    't1h6uX3zAvA8DGkxvizULntja7RS7hZUFYp': 'Zecmine.pro',
    't1hASvMj8e6TXWryuB3L5TKXJB7XfNioZP3': 'Nanopool',
}

MINER_URLS = {
    '2Miners.com': 'https://zec.2miners.com/en',
    'Antpool': 'https://www.antpool.com/poolStats.htm',
    'Bitclub': '',
    'Coinfoundry': 'https://coinfoundry.org/pool/zec',
    'CoinMine.pl': 'https://www2.coinmine.pl/zec/',
    'Coinotron': 'https://www.coinotron.com/coinotron/app?action=ChartNoLogon&span=0&type=C&name=ZEC',
    'DwarfPool': 'https://dwarfpool.com/zec',
    'F2Pool': 'https://www.f2pool.com/',
    'Flypool': 'https://zcash.flypool.org/',
    'MinerGate': 'https://minergate.com/pool-stats/zec',
    'MiningPoolHub': 'https://zcash.miningpoolhub.com/',
    'Nanopool': 'https://zec.nanopool.org/',
    'Nicehash': '',
    'Slush Pool': 'https://slushpool.com/stats/blocks/?c=zec',
    'Suprnova': 'https://zec.suprnova.cc/',
    'uCrypto': 'https://ucrypto.net/pool-blocks/?curr=ZEC',
    'ViaBTC': 'https://pool.viabtc.com/',
    'Waterhole': 'https://zec.waterhole.io/',
    'Zecmine.pro': 'https://zecmine.pro/',
    'Zmine': 'https://forum.z.cash/t/zcash-pool-zmine-closing/2097',
    'Zpool': 'https://zpool.guru/',
}

def named_address(addr):
    if addr in KNOWN_MINERS:
        miner = KNOWN_MINERS[addr]
        if miner not in MINER_URLS:
            raise ValueError('Invalid entry KNOWN_MINERS[%s] = %s' % (addr, miner))
        return miner
    return addr

In [ ]:
import csv
from datetime import datetime

d_height = []
d_time = []
d_sols = []
d_miner = []
d_fr = []
d_others = []

with open('zcash-miner-data.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in reader:
        (miner_addr, miner_value) = row[3].split('~')
        d_height.append(int(row[0]))
        d_time.append(datetime.fromtimestamp(int(row[1])))
        d_sols.append(int(row[2]))
        d_miner.append((named_address(miner_addr), miner_value))
        d_fr.append(row[4])
        d_others.append([other.split('~') for other in row[5].split('/')])

print('Height = %d' % d_height[-1])

In [ ]:
from math import ceil

def bucket_mined_blocks(bucket):
    bucket_mined_blocks = []
    for i in range(ceil(len(d_miner) / bucket)):
        miners = {}
        for j in range(bucket):
            offset = bucket * i + j
            if offset >= len(d_miner):
                break
            addr = d_miner[offset][0]
            try:
                miners[addr] += 1
            except:
                miners[addr] = 1
        bucket_mined_blocks.append(miners)
    # Drop the last bucket, as it isn't full and we aren't plotting percentages
    return bucket_mined_blocks[:-1]

def bucket_sols(bucket):
    bucket_sols = []
    for i in range(ceil(len(d_sols) / bucket) - 1):
        # Sols are backward-looking over 100 blocks
        avg_sols = sum(d_sols[bucket * i:bucket * (i + 1):100]) * 100 / bucket
        bucket_sols.append(avg_sols)
    return bucket_sols

def scale_buckets(buckets, in_scale, out_scale):
    scaled = []
    for i in range(0, len(buckets)):
        scaled_miners = {}
        for m in buckets[i].keys():
            scaled_miners[m] = (buckets[i][m] * out_scale[i]) / in_scale
        scaled.append(scaled_miners)
    return scaled

def bucket_counts(buckets, start, noise):
    bucket_counts = {}
    for i in range(start, len(buckets)):
        for m in buckets[i].keys():
            try:
                bucket_counts[m].append(buckets[i][m])
            except:
                bucket_counts[m] = [0] * (i - start)
                bucket_counts[m].append(buckets[i][m])
        for m in bucket_counts.keys():
            while len(bucket_counts[m]) <= i - start:
                bucket_counts[m].append(0)

    # Sort
    bucket_counts = sorted(bucket_counts.items(), key=lambda item: sum(item[1]), reverse=True)

    # Exclude the noise
    bucket_counts = (b for b in bucket_counts if sum(b[1]) >= noise)
    
    return bucket_counts

In [ ]:
%%opts Area [height=650 width=950 show_legend=True]
%%opts HLine (color='black', line_dash='dashed', line_width=2)
%%opts Overlay [legend_position='bottom_right' show_grid=True]

BUCKET = 1000
START = 0
NOISE = 2
LARGE = 2100

dims = dict(kdims='Date', vdims='Blocks')

xs = [d_time[i * BUCKET] for i in range(START, ceil(len(d_time) / BUCKET))]
miner_areas = []
for (m, c) in bucket_counts(bucket_mined_blocks(BUCKET), START, NOISE):
    # Only show larger miners in the legend
    label = m if sum(c) >= LARGE else ''
    miner_areas.append(hv.Area(zip(xs, c), label=label, **dims))

overlay = miner_areas[0]
for i in range(1, len(miner_areas)):
    overlay = overlay * miner_areas[i]
overlay = overlay.options('Area', fill_alpha=0.5)

(
    hv.Area.stack(overlay)
).relabel(
    'Blocks mined per %d blocks for the %d mining pools and solo miners that have mined at least %d blocks' % (
        BUCKET,
        len(miner_areas),
        NOISE,
    )
)

In [ ]:
%%opts Curve [height=650 width=950]

xs = [x for x in range(len(d_sols))]
a = hv.Curve(zip(xs[::100], d_sols[::100]))
b = hv.Curve(zip(xs[::1000], bucket_sols(1000)))
a * b

In [ ]:
%%opts Area [height=650 width=950 show_legend=True]
%%opts HLine (color='black', line_dash='dashed', line_width=2)
%%opts Overlay [legend_position='top_left' show_grid=True]

BUCKET = 1000
START = 0
NOISE = 25 * 1000000
LARGE = 200 * 1000000

dims = dict(kdims='Date', vdims='Solutions / s')

xs = [d_time[i * BUCKET] for i in range(START, ceil(len(d_time) / BUCKET))]
miner_areas = []
for (m, c) in bucket_counts(
    scale_buckets(
        bucket_mined_blocks(BUCKET),
        BUCKET,
        bucket_sols(BUCKET)), START, NOISE):
    # Only show larger miners in the legend
    label = m if sum(c) >= LARGE else ''
    miner_areas.append(hv.Area(zip(xs, c), label=label, **dims))

overlay = miner_areas[0]
for i in range(1, len(miner_areas)):
    overlay = overlay * miner_areas[i]
overlay = overlay.options('Area', fill_alpha=0.5)

(
    hv.Area.stack(overlay)
).relabel(
    'Network solution rate relative to blocks mined per %d blocks' % (
        BUCKET
    )
)